In [ ]:
import asyncio
import os
import threading
from collections import deque

import msgpack
import websockets

from beamlime.core.serialization import deserialize_data_array


# Thread-safe buffer for latest array
data_buffer = deque(maxlen=1)  # Only keep most recent array


def websocket_worker():
    """Background thread for WebSocket communication"""

    async def connect_websocket():
        uri = "ws://localhost:5555/ws"
        async with websockets.connect(uri) as websocket:
            while True:
                try:
                    data = await websocket.recv()
                    if data:
                        # print('new data')
                        try:
                            arrays = {
                                tuple(k.split('||')): deserialize_data_array(v)
                                for k, v in msgpack.unpackb(data).items()
                            }
                            # data_buffer.append(arrays)
                            provider.update_data(next(iter(arrays.values())))
                        except (msgpack.UnpackException, KeyError) as e:
                            print("Error processing data: %s", e)
                except websockets.ConnectionClosed:
                    print("WebSocket connection closed, attempting to reconnect...")
                    await asyncio.sleep(2)
                    break

    async def keep_alive():
        while True:
            try:
                await connect_websocket()
            except Exception as e:
                print(f"WebSocket error: {e}")
                await asyncio.sleep(2)

    asyncio.run(keep_alive())

In [ ]:
%matplotlib widget
import plopp
import ipywidgets as ipw
import scipp as sc
import numpy as np
from dataclasses import dataclass
from numpy.random import default_rng

rng = default_rng(seed=1234)


def make_random_data() -> sc.DataArray:
    nrow = 1000
    x = sc.array(dims=['row'], unit='m', values=(rng.random(nrow)))
    y = sc.array(dims=['row'], unit='m', values=(rng.random(nrow)))
    data = sc.ones(dims=['row'], unit='K', shape=[nrow])
    data.values += 0.1 * rng.random(nrow)
    return sc.DataArray(data=data, coords={'x': x, 'y': y}).hist(x=100, y=100) * 10

In [ ]:
import threading
import time


@dataclass
class DataHandle:
    value: sc.DataArray


class DataProvider:
    def __init__(self, value: sc.DataArray):
        self._handle = DataHandle(value)
        self._node = plopp.Node(self._handle)

        @plopp.node
        def get_value(handle: DataHandle) -> sc.DataArray:
            return handle.value

        self.value_node = get_value(self._node)

    def update_data(self, value: sc.DataArray):
        self._handle.value = value
        self._node.notify_children(message='hi')


provider = DataProvider(None)


threading.Thread(target=websocket_worker, daemon=True).start()

vmax_slider = ipw.IntRangeSlider(
    max=1000, description='vmax', layout={'width': '400px'}, value=(0, 1000)
)
vmax_node = plopp.widget_node(vmax_slider)


@plopp.node
def clip_data(da: sc.DataArray, clim: tuple[int, int]) -> sc.DataArray:
    vmin, vmax = clim
    da.data = sc.where(
        (da.data > sc.scalar(vmax, unit=da.unit))
        | (da.data < sc.scalar(vmin, unit=da.unit)),
        sc.scalar(np.nan, unit=da.unit),
        da.data,
    )
    return da


clipped = clip_data(provider.value_node, clim=vmax_node)

In [ ]:
image = plopp.imagefigure(vmin=1, vmax=200, cbar=True, aspect='equal')
ipw.VBox([image])

In [ ]:
provider.value_node.add_view(image)